In [21]:
import tensorflow as tf
import pandas as pd
import os
import sys
sys.path.append('/Users/hoyoung/Desktop/pycharm_work/korean_grammar_corrector/bin')
sys.path.append('/Users/hoyoung/Desktop/pycharm_work/korean_grammar_corrector/utils')

from tensorflow_preprocess import *
from model.transformer_model import *
from model.predict import *
from keras_model.keras_tensorflow_cust import *

In [2]:
df = pd.read_csv('../../../data/train/korean_corpus_test_20221201.csv', names=['src', 'tgt'])

In [3]:
df.head()

,src,tgt
0,연방정부느 니와 함께 현재 판타나우에서 소방대를 통해 이뤄지는 진화자거블 확때하기 ...,연방정부는 이와 함께 현재 판타나우에서 소방대를 통해 이뤄지는 진화작업을 확대하기 ...
1,"누르 에딘박싸는 ""현재 이슬람방물관과 콥틱빵물과늬 히귀 소장푸메 라베를 부착쭝이며 ...","누르 에딘박사는 ""현재 이슬람박물관과 콥틱박물관의 희귀 소장품에 라벨을 부착중이며 ..."
2,"델타 규치근 다닐-층 뉴럴 레트워크드르 뤼해 설계되얻끼 때무네, 그거싀 일반화이 녁...","델타 규칙은 단일-층 뉴럴 네트워크들을 위해 설계되었기 때문에, 그것의 일반화인 역..."
3,이러한 실처늘 가능하게 호근 성공저그로 만든 멷 까지 구조저 교건드를 지저캐 둘 피...,이러한 실천을 가능하게 혹은 성공적으로 만든 몇 가지 구조적 요건들을 지적해 둘 필...
4,"궁내에 세책쩌미 등장한 거시 화긴된다면, 세계 세챙 문화사를 로코 볼 때 이른 시기...","국내에 세책점이 등장한 것이 확인된다면, 세계 세책 문화사를 놓고 볼 때 이른 시기..."


In [22]:
model_70_path = '../../../checkpoint_maxlength70/cp.ckpt'
tokenizer_70_path = '../../../tokenizer/tokenizer_maxlength_70.tok'
tokenizer_70 = load_tokenizer(tokenizer_70_path)

model_70 = Transformer_Model(vocab_size=tokenizer_70.vocab_size+2, d_model=128, num_layers=4, num_heads=4, dff=256, dropout=0.1, name='test')
model_70.load_weights(model_70_path)

In [23]:
model_keras_path = '../../../keras_checkpoint_20221213/keras_new_model_test'
tokenizer_keras_path = '../../../keras_checkpoint_20221213/tokenizer_maxlength_70.tok'

tokenizer_keras = load_tokenizer(tokenizer_keras_path)

model_keras = Model(vocab_size=tokenizer_keras.vocab_size+2, d_model=128, num_layers=4, num_heads=4, dff=256, dropout=0.1, name='test')
model_keras.load_weights(model_keras_path)

In [24]:
model_70.summary()

Model: "test"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inputs (InputLayer)            [(None, None)]       0           []                               
                                                                                                  
 dec_inputs (InputLayer)        [(None, None)]       0           []                               
                                                                                                  
 enc_padding_mask (Lambda)      (None, 1, 1, None)   0           ['inputs[0][0]']                 
                                                                                                  
 encoder (Functional)           (None, None, 128)    1577600     ['inputs[0][0]',                 
                                                                  'enc_padding_mask[0][0]']    

In [25]:
model_keras.summary()

Model: "test"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inputs (InputLayer)            [(None, None)]       0           []                               
                                                                                                  
 dec_inputs (InputLayer)        [(None, None)]       0           []                               
                                                                                                  
 encoder (Functional)           (None, None, 128)    1375872     ['inputs[0][0]']                 
                                                                                                  
 decoder (Functional)           (None, None, 128)    1639936     ['dec_inputs[0][0]',             
                                                                  'encoder[0][0]']             

In [32]:
sent = '안뇽하셂요'
predict_70 = Predictor(70, model_70, tokenizer_70)
predict_keras = Predictor(50, model_keras, tokenizer_keras)

In [33]:
predict_70.predict(df['src'][0], model_70, debug=True)

Input: 연방정부느 니와 함께 현재 판타나우에서 소방대를 통해 이뤄지는 진화자거블 확때하기 위한 재정지원 방안도 검토하고 읻따.
Output: 연방정부는 이와 함께 현재 판타나우에서 소방대를 통해 이뤄지는 진화작업을 확대하기 위한 재정지원 방안도 검토하고 있다 .


'연방정부는 이와 함께 현재 판타나우에서 소방대를 통해 이뤄지는 진화작업을 확대하기 위한 재정지원 방안도 검토하고 있다 .'

In [34]:
predict_keras.predict(df['src'][0], model_70, debug=True)

Input: 연방정부느 니와 함께 현재 판타나우에서 소방대를 통해 이뤄지는 진화자거블 확때하기 위한 재정지원 방안도 검토하고 읻따.
Output: 연방정부느 니와 함께 현재 판타나우에서 극복0대를 를 이뤄지는 진화자거블 확때하기 위한 재정지원 방안도 검토하고 있다 .혁신 .이 대를 를 이뤄지는 진화자거블 확때으며이 재정지원 방안


'연방정부느 니와 함께 현재 판타나우에서 극복0대를 를 이뤄지는 진화자거블 확때하기 위한 재정지원 방안도 검토하고 있다 .혁신 .이 대를 를 이뤄지는 진화자거블 확때으며이 재정지원 방안'

###### Keras 모델 수정이 필요함.

In [11]:
# sampling
from random_sample_util import *

In [12]:
test = random_sampling(df, random_seed=123, sample_num=100, data='test')

In [15]:
predict_70 = Predictor(70, model_70, tokenizer_70)
predict_24 = Predictor(24, model_24, tokenizer_24)

pred_70 = test['src'].apply(lambda x: predict_70.predict(x, model_70))
pred_24 = test['src'].apply(lambda x: predict_24.predict(x, model_24))

In [16]:
test['predict_70'] = pred_70
test['predict_24'] = pred_24

In [32]:
test.to_excel('../../../data/testsets_sample100.xlsx', index=False)

In [57]:
num_test_text = '연방정부느 니와 함께 현재 판타나우에서 A소방대를 통해 이뤄지는 진화자거블 2배 확때하기 위한 재정지원 방안도 검토하고 읻따'

In [17]:
test = '안녕하세요. 반갑습니다.'

In [18]:
# 숫자와 영문을 제외한 데이터로만 훈련을 해서 숫자 또는 영문이 들어간 경우 비정상적으로 교정됨.
predict_70.predict(test, model_70, debug=True)

Input: 안녕하세요. 반갑습니다.
Output: 안녕하세요 . 반갑습니다 .


'안녕하세요 . 반갑습니다 .'

In [59]:
from bleu_score import *

In [60]:
bleu = BleuModule(weight=(0.25, 0.25, 0.25, 0.25))

In [69]:
test['predict_ref'] = test.apply(lambda x: bleu.calculate(x['tgt'], x['src']), axis=1)
test['predict_70_bleu'] = test.apply(lambda x: bleu.calculate(x['tgt'], x['predict_70']), axis=1)
test['predict_24_bleu'] = test.apply(lambda x: bleu.calculate(x['tgt'], x['predict_24']), axis=1)

In [70]:
test.to_excel('../../../data/testsets_sample100.xlsx', index=False)

- 학습 절차 개선
- 학습된 모델 로드 -> 인터페이스화
- 인퍼런스 서버 (인터페이스, 서버, Restful)

1. 리펙토링 (api class) -> 인퍼런스 클래스화
2. Restful api 
